In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 64
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, 
                                   width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'data\ASL_Dataset',  
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training')

val_generator = train_datagen.flow_from_directory(
    'data\ASL_Dataset', 
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

 
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(train_generator, validation_data=val_generator, epochs=10)


model.save('asl_model.h5')


Found 132554 images belonging to 28 classes.
Found 33132 images belonging to 28 classes.


c:\Users\Gpentela\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


c:\Users\Gpentela\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4143/4143 ━━━━━━━━━━━━━━━━━━━━ 1053s 254ms/step - accuracy: 0.5915 - loss: 1.3043 - val_accuracy: 0.9569 - val_loss: 0.1278
Epoch 2/10
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 296s 71ms/step - accuracy: 0.9542 - loss: 0.1351 - val_accuracy: 0.9791 - val_loss: 0.0635
Epoch 3/10
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 304s 73ms/step - accuracy: 0.9755 - loss: 0.0753 - val_accuracy: 0.9824 - val_loss: 0.0559
Epoch 4/10
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 305s 74ms/step - accuracy: 0.9833 - loss: 0.0554 - val_accuracy: 0.9875 - val_loss: 0.0375
Epoch 5/10
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 304s 73ms/step - accuracy: 0.9864 - loss: 0.0426 - val_accuracy: 0.9857 - val_loss: 0.0603
Epoch 6/10
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 300s 72ms/step - accuracy: 0.9891 - loss: 0.0358 - val_accuracy: 0.9853 - val_loss: 0.0455
Epoch 7/10
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 305s 74ms/step - accuracy: 0.9904 - loss: 0.0318 - val_accuracy: 0.9921 - val_loss: 0.0262
Epoch 8/10
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 312s 75ms/step - accuracy: 0.9911 - 

In [3]:
import json

# Save class labels to a JSON file after defining train_generator
with open('class_labels.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

# You can add this after model.fit()
model.save('asl_model.h5')


In [5]:
import cv2
import numpy as np
import math

# Webcam feed setup
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    # Flip the frame horizontally to create a mirror effect
    frame = cv2.flip(frame, 1)

    # Define the region of interest (ROI) for hand detection (optional)
    roi = frame[100:400, 100:400]

    # Convert the frame to grayscale
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)

    # Apply thresholding to get binary image
    _, thresholded = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours from the binary image
    contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour (which should be the hand)
    if len(contours) > 0:
        max_contour = max(contours, key=cv2.contourArea)

        # Get the convex hull of the largest contour
        hull = cv2.convexHull(max_contour)

        # Draw contours and hull
        cv2.drawContours(roi, [max_contour], -1, (0, 255, 0), 2)
        cv2.drawContours(roi, [hull], -1, (0, 0, 255), 2)

        # Get convexity defects (the points between fingers)
        hull_indices = cv2.convexHull(max_contour, returnPoints=False)
        defects = cv2.convexityDefects(max_contour, hull_indices)

        if defects is not None:
            count_fingers = 0

            # Loop over all detected defects
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i][0]
                start = tuple(max_contour[s][0])
                end = tuple(max_contour[e][0])
                far = tuple(max_contour[f][0])

                # Calculate the angle between the start, end, and far points (to detect fingers)
                a = math.dist(start, end)
                b = math.dist(far, start)
                c = math.dist(far, end)

                angle = math.acos((b**2 + c**2 - a**2) / (2 * b * c))

                # If the angle is less than 90 degrees, it's likely between two fingers
                if angle <= math.pi / 2:
                    count_fingers += 1
                    cv2.circle(roi, far, 5, (0, 0, 255), -1)

            # Display the number of fingers detected
            cv2.putText(frame, f'Fingers: {count_fingers + 1}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Finger Detection', frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
